In [26]:
import pandas as pd
import os
from datetime import datetime as dt
import zipfile
import glob
import shutil

In [27]:
years = []

for file in os.listdir(os.getcwd()):
    if file.endswith(".zip"):
        years.append(file)
yy = []        

def yearExtractor(y):
    with zipfile.ZipFile(y,'r') as myZip:
        newy = y
        newy = newy.removesuffix('.zip')
        yy.append(newy)
        myZip.extractall(f"{newy}")   
        
def monthExtractor(m):
    dir_name = os.path.join(os.getcwd()+ f"/{m}")
    extension = ".zip"

    os.chdir(dir_name) # change directory from working dir to dir with files

    for item in os.listdir(dir_name): # loop through items in dir
        if item.endswith(extension): # check for ".zip" extension
            file_name = os.path.abspath(item) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(dir_name) # extract file to dir
            zip_ref.close() # close file
            os.remove(file_name) # delete zipped file
    os.chdir("../")
        
        
x = list(map(yearExtractor,years))
x2 = list(map(monthExtractor,yy))

path = os.path.join(os.getcwd()+"/Stocks")

if not (os.path.exists(path)):
    os.makedirs(path)
liY = [] 
path = os.listdir()
for year in path:
    if (os.path.isdir(year) and os.path.dirname(year)!='Stocks') :
        liY.append(year)       

for year in liY:
    liM=[]
    for month in os.listdir(os.path.join((os.getcwd()+"/"+year))):
        path = os.path.join((os.getcwd()+"/"+year+"/"+month))
        if (os.path.isdir(path)):
            liM.append(month)
    print(year,liM)       
    frame = pd.DataFrame()
    for month in liM:
        path = os.getcwd()+f'/{year}/{month}'
        all_files = glob.glob(path + "/*.csv")
        li=[]
        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=0,parse_dates=['Date'],dayfirst=True)
            li.append(df)    
        frame = pd.concat(li, axis=0, ignore_index=True)
        frame = frame.dropna()
        frame["Date"]=pd.to_datetime(frame["Date"])
        frame["Time"]=pd.to_datetime(frame["Time"],format="%H:%M:%S").dt.time
        frame.sort_values(by=["Ticker","Date"],inplace=True,ignore_index=True)
        unames = frame["Ticker"].unique()
        for curr in unames:
            print(os.path.exists(os.path.join(f"{os.getcwd()}/Stocks/{curr}.csv")))
            if os.path.exists(os.path.join(f"{os.getcwd()}/Stocks/{curr}.csv")):
                print(curr)
                df = pd.DataFrame()
                for rows in range(len(frame)):
                    if frame.loc[rows,"Ticker"]==curr:
                        df= df.append(frame.iloc[rows],ignore_index=True)
                tmp = pd.read_csv(os.path.join(f"{os.getcwd()}/Stocks/{curr}.csv"),parse_dates=['Date'],dayfirst=True)
                tmp2 = pd.concat([df,tmp],axis=0,ignore_index=True)
                for rows in range(len(tmp2)):
                    tmp2.loc[rows,"Date"] = tmp2.loc[rows,"Date"].strftime("%d/%m/%y")
                tmp2.sort_values(by=['Date',"Time"],inplace=True)
                tmp2.to_csv(f"{os.getcwd()}/Stocks/{curr}.csv",index=False)
            else:
                print(curr)
                dfn = pd.DataFrame()
                for rows in range(len(frame)):
                    if frame.loc[rows,"Ticker"]==curr:
                        dfn=dfn.append(frame.iloc[rows],ignore_index=True)
                dfn.sort_values(by=["Date","Time"],inplace=True)
                dfn.to_csv(f"{os.getcwd()}/Stocks/{curr}.csv",index=False)
    shutil.rmtree(os.path.join(os.getcwd()+f"/{year}"))                